<a href="https://colab.research.google.com/github/fschwartzer/DAI/blob/main/BOXES_Exerc%C3%ADcio_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importação das bibliotecas
import sys
!{sys.executable} -m pip install gradio
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gradio as gr
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#Carregar dados

fileDownloaded = drive.CreateFile({'id':'1ioRyFX98kPTXjM7H2r4BABr27UrcTSlo'})
fileDownloaded.GetContentFile('new_X.csv')
X = pd.read_csv('new_X.csv', header=None)

fileDownloaded = drive.CreateFile({'id':'18KPR5uYCjhOCUCPN-WARx3q236-piF78'})
fileDownloaded.GetContentFile('y.csv')
y = pd.read_csv('y.csv', header=None)

#Normalizar dados
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
input_scaler = scaler_X.fit(X)
output_scaler = scaler_y.fit(y)
X_norm = input_scaler.transform(X)
y_norm = output_scaler.transform(y)
new_X = X_norm
new_y = np.ravel(y_norm)

#Dividir dados em base de treino e de teste
train_dataset, test_dataset, train_labels, test_labels = train_test_split(new_X, new_y, test_size=0.2, random_state=42)

#Transformar dados em DMatrix
dtrain = xgb.DMatrix(train_dataset, train_labels)
dtest = xgb.DMatrix(test_dataset, test_labels)

#Modelo
num_boost_round = 999
params = {'max_depth': 5,
          'min_child_weight': 9,
          'eta': 0.1,
          'subsample': 1.0,
          'colsample_bytree': 1.0,
          'objective': 'reg:squarederror',
          'eval_metric': 'mae'}
model = xgb.train(params,
                  dtrain,
                  num_boost_round=num_boost_round,
                  evals=[(dtest, "Test")],
                  early_stopping_rounds=10)

num_boost_round = model.best_iteration + 1

best_model = xgb.train(params,
                       dtrain,
                       num_boost_round=num_boost_round,
                       evals=[(dtest, "Test")])
#Fazer as inferências
def predict_box(Atotal, Residencial, Coberta, Idade, DF, B, C, D, E, X, Y):
    df = pd.DataFrame.from_dict({'Atotal': np.log([Atotal]),
                                 'Residencial': [Residencial],
                                 'Coberta': [Coberta],
                                 'Idade': [Idade],
                                 'DF': [DF],
                                 'B': [B],
                                 'C': [C],
                                 'D': [D],
                                 'E': [E],
                                 'X': [X], 
                                 'Y': [Y]})
    df = input_scaler.transform(df)
    df = xgb.DMatrix(df)
    pred = best_model.predict(df)
    pred = output_scaler.inverse_transform(np.array(pred).reshape(-1, 1))
    pred = np.exp(pred).tolist()     
    return f"""Valor Total do box: R${round(pred[0][0])}"""

#Definir os campos de inserção de dados
Atotal = gr.inputs.Number(default = 20., label="Área Total")
Residencial = gr.inputs.Number(default = 1, label="Residencial")
Coberta = gr.inputs.Number(default = 1, label="Coberta")
Idade = gr.inputs.Number(default = 5, label="Idade")
DF = gr.inputs.Number(default = 1, label="Divisão Fiscal")
B = gr.inputs.Number(default = 0, label="Padrão Construtivo B")
C = gr.inputs.Number(default = 1, label="Padrão Construtivo C")
D = gr.inputs.Number(default = 0, label="Padrão Construtivo D")
E = gr.inputs.Number(default = 0, label="Padrão Construtivo E")
X = gr.inputs.Number(default =274800., label="Longitude")
Y = gr.inputs.Number(default =1.662188e+06, label="Latitude")

#Criar interface do aplicativo
gr.Interface(predict_box, [Atotal, Residencial, Coberta, Idade, DF, B, C, D, E, X, Y], "label", live=False).launch(share=False, debug=True)

     |████████████████████████████████| 3.4 MB 5.1 MB/s 
     |████████████████████████████████| 1.9 MB 40.7 MB/s 
     |████████████████████████████████| 206 kB 48.1 MB/s 
     |████████████████████████████████| 961 kB 48.5 MB/s 
     |████████████████████████████████| 3.0 MB 31.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=8920a4b79d6903387037d7b0851d7521967a190474bdc06235b9c7b262de8da1
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3371 sha256=ed9ee6cb1f3f9a2f729997301a881c1317ef30d3f08a82e8ef609130e8bdff02
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster
[0]	Test-mae:0.103296
Will train until Test-mae hasn't improved in 10 rounds.
[1